In [10]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 16 00:34:36 2023

@author: dagom EJEMPLO DEL BANCO !!!!!!
"""
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import string
import re

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [11]:
## LECTURA DE DATOS

bank=pd.read_csv('bank-full.csv', sep=';')

variables=bank.columns
print(variables)

X=bank.iloc[:,0:16]
y=pd.DataFrame(bank["y"])

y.describe()
y.value_counts()
## clase del no 39922/45211 ## muy balanceado

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')


y  
no     39922
yes     5289
Name: count, dtype: int64

In [12]:
## Paso 1. vemos cuantos missing tenemos en cada variable ##
bank.isnull().sum()

# Paso 2. Para poder aplicar SVM las variables deben ser numericas y a ser posibles estandarizadas
from sklearn import preprocessing

col_cat = bank[['job', 'marital','education','default', 'housing','loan','contact','y']]
col_num = bank[['age', 'balance', 'day','duration','pdays','previous']]


scaler = preprocessing.StandardScaler().fit(col_num)
col_num_standarizada = scaler.transform(col_num)

col_num_standarizada.mean(axis=0)
col_num_standarizada.std(axis=0)

col_num_standarizada = pd.DataFrame(col_num_standarizada)


bank_depurada = pd.concat([col_num_standarizada, col_cat],  axis=1) ## aunque asi se pierden los nombres de las variables

## recuperamos los nombres y tenemos  nuestra base de datos depurada
bank_depurada=bank_depurada.set_axis(['age', 'balance', 'day' ,'duration','pdays','previous' , 'job', 'marital','education','default', 'housing','loan','contact','y'], axis=1)


## Paso 3. Variables categoricas las pasamos a dummies

bank_depurada_dummies = pd.get_dummies(bank_depurada,columns=['job', 'marital','education','default', 'housing','loan','contact','y'], drop_first= True)
bank_depurada_dummies.head()


X=bank_depurada_dummies.iloc[:,0:26]
y=pd.DataFrame(bank_depurada_dummies["y_yes"])

y.describe()
y.value_counts()


########### hacemos clasificacion con SVM ####
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

[X_train, X_test, y_train, y_test] = train_test_split(X, y, test_size = 0.30, random_state = 101)


###

# train the model on train set
model = SVC(kernel='linear')
model.fit(X_train, y_train)

# print prediction results
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

cm = confusion_matrix(y_test, predictions)
print(cm)  ## ojo a esta matriz de confusion !!!!!
accuracy=(cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+ cm[1,0]+cm[0,0])
print('accuracy' , accuracy)

              precision    recall  f1-score   support

       False       0.88      1.00      0.94     11967
        True       0.00      0.00      0.00      1597

    accuracy                           0.88     13564
   macro avg       0.44      0.50      0.47     13564
weighted avg       0.78      0.88      0.83     13564

[[11967     0]
 [ 1597     0]]
accuracy 0.882261869654969


In [ ]:
#############################
################## smooooote
#############################




## busqueda de parametros para el caso lineal ###
from sklearn.model_selection import GridSearchCV

# definimos los rangos de los parametros
param_grid_lineal = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 1, 2, 5, 10, 100, 1000] }
grid = GridSearchCV(SVC(kernel='linear'), param_grid_lineal, refit = True, cv=5, verbose = 3)

# ENTRENAMOS EN TRAIN Y BUSCAMOS EN TRAIN
resultados = grid.fit(X_train, y_train)

#############################

grid_predictions = grid.predict(X_test)

# print classification report
print(classification_report(y_test, grid_predictions))
cm = confusion_matrix(y_test, grid_predictions)
print(cm)
accuracy=(cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+ cm[1,0]+cm[0,0])
print('accuracy' , accuracy)


Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5] END .............................C=0.1;, score=0.883 total time=   9.9s
[CV 2/5] END .............................C=0.1;, score=0.883 total time=   8.2s
[CV 3/5] END .............................C=0.1;, score=0.883 total time=  10.0s
[CV 4/5] END .............................C=0.1;, score=0.883 total time=  10.4s
[CV 5/5] END .............................C=0.1;, score=0.883 total time=   8.3s
[CV 1/5] END .............................C=0.2;, score=0.883 total time=  10.9s
[CV 2/5] END .............................C=0.2;, score=0.883 total time=  10.5s
[CV 3/5] END .............................C=0.2;, score=0.883 total time=   9.7s
[CV 4/5] END .............................C=0.2;, score=0.883 total time=  11.4s
[CV 5/5] END .............................C=0.2;, score=0.883 total time=  10.7s
[CV 1/5] END .............................C=0.3;, score=0.883 total time=  11.5s
[CV 2/5] END .............................C=0.3;

In [13]:


from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_train, y_train)


y_res.describe()
y_res.value_counts()


model = SVC(kernel='linear')
model.fit(X_res, y_res)

# print prediction results
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

cm = confusion_matrix(y_test, predictions)
print(cm)
accuracy=(cm[0,0]+cm[1,1])/(cm[0,1]+cm[1,1]+ cm[1,0]+cm[0,0])
print('accuracy' , accuracy)


              precision    recall  f1-score   support

       False       0.96      0.81      0.88     11967
        True       0.35      0.75      0.47      1597

    accuracy                           0.80     13564
   macro avg       0.65      0.78      0.68     13564
weighted avg       0.89      0.80      0.83     13564

[[9687 2280]
 [ 393 1204]]
accuracy 0.8029342376879977
